In [47]:
from Dataset import SoccerActionDataset
from torch.utils.data import Dataset, DataLoader, random_split

In [48]:
import torch
import torch.nn as nn
from pytorchvideo.models.hub import x3d_s

In [49]:
ds = SoccerActionDataset("./data/720p/train", "Labels-ball.json", num_frames=16)

frames, label = ds[0]
print(frames.shape)

train_ratio = 0.8
train_size = int(train_ratio * len(ds))
test_size = len(ds) - train_size

train_ds, test_ds = random_split(ds, [train_size, test_size])

train_loader = DataLoader(train_ds, batch_size = 8, shuffle = True, num_workers = 4, pin_memory = True)
test_loader = DataLoader(test_ds, batch_size = 8, shuffle = True, num_workers = 4, pin_memory = True)

torch.Size([3, 16, 112, 112])


In [50]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [51]:
num_classes = len(ds.label_to_idx)

In [52]:
model = x3d_s(pretrained=True)

In [53]:
in_features = model.blocks[-1].proj.in_features
model.blocks[-1].proj.in_features = nn.Linear(in_features, num_classes)
model.to(device=device)

Net(
  (blocks): ModuleList(
    (0): ResNetBasicStem(
      (conv): Conv2plus1d(
        (conv_t): Conv3d(3, 24, kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), bias=False)
        (conv_xy): Conv3d(24, 24, kernel_size=(5, 1, 1), stride=(1, 1, 1), padding=(2, 0, 0), groups=24, bias=False)
      )
      (norm): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
    )
    (1): ResStage(
      (res_blocks): ModuleList(
        (0): ResBlock(
          (branch1_conv): Conv3d(24, 24, kernel_size=(1, 1, 1), stride=(1, 2, 2), bias=False)
          (branch2): BottleneckBlock(
            (conv_a): Conv3d(24, 54, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
            (norm_a): BatchNorm3d(54, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act_a): ReLU()
            (conv_b): Conv3d(54, 54, kernel_size=(3, 3, 3), stride=(1, 2, 2), padding=(1, 1, 1), groups=54, bias=False)
            (nor

In [55]:
x = torch.randn(2, 3, 16, 182, 182).to(device)
y = model(x)    
print(y.shape)  

torch.Size([2, 400])


In [58]:
from collections import Counter
label_counts = Counter(s['label'] for s in ds.samples)
label_counts

Counter({'HIGH PASS': 2748,
         'HEADER': 2387,
         'DRIVE': 13689,
         'PLAYER SUCCESSFUL TACKLE': 204,
         'PASS': 15955,
         'CROSS': 1059,
         'OUT': 1983,
         'THROW IN': 1269,
         'SHOT': 600,
         'BALL PLAYER BLOCK': 762})

In [59]:
import numpy as np
class_counts = np.array([label_counts[ds.idx_to_label[i]] for i in range(num_classes)])
class_counts

array([  762,  1059, 13689,  2387,  2748,  1983, 15955,   204,   600,
        1269])

In [65]:
class_weights =  1 / class_counts
class_weights /= class_weights.sum() * num_classes
class_weights

array([0.01189128, 0.00855633, 0.00066193, 0.00379604, 0.00329736,
       0.00456942, 0.00056792, 0.04441741, 0.01510192, 0.00714039])

In [66]:
class_weights = torch.FloatTensor(class_weights).to(device)

In [69]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)